step 1- Load libraries

In [4]:
# ===== Step 1: Imports, seeds, and GPU setup =====

# Standard libraries
import os, random                              # OS utilities and reproducibility
import numpy as np                             # Numerical arrays

# TensorFlow / Keras
import tensorflow as tf                        # Main DL framework

# Keras layers & utilities
from tensorflow.keras.layers import (Input, SeparableConv2D, DepthwiseConv2D, BatchNormalization,
                                     Activation, AveragePooling2D, Dropout, Dense, Add, Lambda,
                                     GlobalAveragePooling2D, GlobalAveragePooling1D,
                                     MultiHeadAttention, Reshape, LayerNormalization, Conv2D)
from tensorflow.keras.models import Model      # Functional model API
from tensorflow.keras.optimizers import Adam   # Optimizer
from tensorflow.keras.losses import BinaryCrossentropy  # Binary classification loss
from tensorflow.keras.metrics import Accuracy  # Accuracy metric
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint  # Training callbacks
from tensorflow.keras import backend as K      # Keras backend for static shape queries

# Scikit-learn utilities
from sklearn.model_selection import StratifiedKFold                  # Stratified 5-fold CV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score  # Metrics

# Plotting (optional)
import matplotlib.pyplot as plt               # For charts (optional)

# Reproducibility
SEED = 42                                     # Fixed seed
np.random.seed(SEED)                          # Seed NumPy
tf.random.set_seed(SEED)                      # Seed TensorFlow
random.seed(SEED)                             # Seed Python

# Optional: make TensorFlow gentle with GPU memory
gpus = tf.config.list_physical_devices('GPU') # Detect GPUs
print("TensorFlow:", tf.__version__)          # Show TF version
print("GPUs:", gpus)                          # Show detected GPUs
if gpus:                                      # If GPU(s) are present
    for g in gpus:                            # Iterate each GPU
        try:
            tf.config.experimental.set_memory_growth(g, True)  # Enable memory growth
        except Exception as e:
            print("GPU memory growth not set:", e)             # Warn if not possible


TensorFlow: 2.20.0
GPUs: []


Step 2- Load arrays

In [5]:
# ===== Step 2: Load the arrays in the given path (no normalization) =====

# Exact Windows paths you provided (raw strings to avoid backslash escapes)
eeg_path    = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data\final_eeg.npy"
labels_path = r"C:\Self Learning\Research Papers\UOW Research Papers\eSport Players with EEG Data\EEG Data\final eeg data\final_eeg_labels.npy"

print("Loading EEG data...")                 # Status print
X = np.load(eeg_path)                        # Load EEG data, expected shape (N, 14, 200)
y = np.load(labels_path)                     # Load labels, expected shape (N,) or (N,1)
print("Loaded.")                             # Confirm loaded

print("Raw X shape:", X.shape, "| dtype:", X.dtype)  # Inspect EEG tensor
print("Raw y shape:", y.shape, "| dtype:", y.dtype)  # Inspect labels

y = np.squeeze(y).astype(int)                # Ensure labels are 1D int array
print("y squeezed shape:", y.shape, "| unique labels:", np.unique(y))  # Show label distribution

# Sanity checks on shapes and labels
assert X.ndim == 3, f"Expected X to be (N,14,200). Got {X.shape}"      # Check rank
assert X.shape[1] == 14 and X.shape[2] == 200, "Expected 14 electrodes and 200 samples per segment."  # Check dims
assert set(np.unique(y)).issubset({0,1}), "Labels must be {0,1}."      # Binary check

# Add channel dimension for Keras Conv2D: (N, 14, 200, 1)
X = X[..., np.newaxis]                         # Append channel dim
print("X after adding channel dim:", X.shape)  # Verify final input shape


Loading EEG data...
Loaded.
Raw X shape: (4770, 14, 200) | dtype: float32
Raw y shape: (4770,) | dtype: int64
y squeezed shape: (4770,) | unique labels: [0 1]
X after adding channel dim: (4770, 14, 200, 1)


Step 3: Normalising Data

In [6]:
# ===== Step 3: Normalization helper (per-fold; compute stats on train only) =====

def normalize_per_fold(X_train, X_val, eps=1e-8):
    """
    Normalizes X_train and X_val using mean/std computed from X_train only.
    Broadcasting shape is (1, 14, 1, 1), i.e., per-electrode normalization.
    """
    # Step 1: Compute mean & std from the training set only
    train_mean = np.mean(X_train, axis=(0, 2, 3), keepdims=True)   # -> (1, 14, 1, 1)
    train_std  = np.std( X_train, axis=(0, 2, 3), keepdims=True)   # -> (1, 14, 1, 1)

    # Safety: avoid division by (near) zero
    train_std = np.where(train_std < eps, 1.0, train_std)

    # Step 2: Apply normalization using training mean & std
    X_train_norm = ((X_train - train_mean) / train_std).astype(np.float32)
    X_val_norm   = ((X_val   - train_mean) / train_std).astype(np.float32)

    print("Per-fold normalization done.",
          "train_mean shape:", train_mean.shape, "train_std shape:", train_std.shape)

    return X_train_norm, X_val_norm


Step 4: Prepare 5-fold CV

In [7]:
# ===== Step 4: Prepare the data to perform 5-fold cross validation =====

N_SPLITS = 5                                   # Number of CV folds
skf = StratifiedKFold(n_splits=N_SPLITS,       # Stratified (preserves class ratios)
                      shuffle=True,            # Shuffle before splitting
                      random_state=SEED)       # Reproducible splits

print(f"Prepared StratifiedKFold with {N_SPLITS} splits.")  # Status
print("Total samples:", X.shape[0])                         # Dataset size


Prepared StratifiedKFold with 5 splits.
Total samples: 4770


Step 5: Model, compile, callbacks

In [8]:
# ===== Step 5: Define AFNet (your code, only shape edits + Keras-safe ops) =====

def SpatialAttention(x):
    """Spatial attention to weight electrodes; uses Keras Reshape/Multiply (no raw tf ops on KerasTensors)."""
    num_electrodes = K.int_shape(x)[1]                        # Static electrode count (14)
    attn = GlobalAveragePooling2D()(x)                        # Global pool -> (batch, channels)
    attn = Dense(64, activation='relu')(attn)                 # Dense to learn attention features
    attn = Dense(num_electrodes, activation='sigmoid')(attn)  # One weight per electrode
    attn = Reshape((num_electrodes, 1, 1))(attn)              # Reshape for broadcast across time & channels
    x = tf.keras.layers.Multiply()([x, attn])                 # Apply attention weights
    return x                                                  # Return attended feature maps

def TransformerBlock(x, num_heads=4, key_dim=64, ff_dim=128, dropout_rate=0.1):
    """Transformer encoder block: MHA + FFN with residuals and layer norms."""
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)  # Self-attention
    attn_output = Dropout(dropout_rate)(attn_output)                               # Dropout
    x = Add()([x, attn_output])                                                   # Residual
    x = LayerNormalization()(x)                                                   # LayerNorm

    ff = Dense(ff_dim, activation='relu')(x)                                      # Feed-forward (expand)
    ff = Dropout(dropout_rate)(ff)                                                # Dropout
    ff = Dense(x.shape[-1])(ff)                                                   # Project back
    x = Add()([x, ff])                                                            # Residual
    x = LayerNormalization()(x)                                                   # LayerNorm
    return x                                                                      # Return block output

def EEGNet_SpatialTransformer(input_shape=(14, 200, 1), dropout_rate=0.5, num_heads=4, ff_dim=128):
    """AFNet adapted for (14,200,1); keeps your structure, minimal shape handling for residual add."""
    print("Building model with input_shape:", input_shape)                         # Trace

    inputs = Input(shape=input_shape)                                              # Input tensor

    # Depthwise Separable Convolution (temporal)
    x = SeparableConv2D(32, (1, 5), padding='same', use_bias=False)(inputs)       # First sep conv
    x = BatchNormalization()(x)                                                    # BN
    x = Activation('relu')(x)                                                      # ReLU
    print("After sepconv1:", x.shape)                                              # Shape trace

    # Spatial Attention
    x = SpatialAttention(x)                                                        # Attention over electrodes
    print("After SpatialAttention:", x.shape)                                      # Shape trace

    # Depthwise Spatial Convolution across electrodes (change 61->14)
    x = DepthwiseConv2D((14, 1), use_bias=False, depth_multiplier=2, padding='valid')(x)  # Collapse electrode dim
    x = BatchNormalization()(x)                                                    # BN
    x = Activation('relu')(x)                                                      # ReLU
    print("After depthwise spatial:", x.shape)                                     # Shape trace (electrode dim -> 1)

    # Residual branch from inputs via the same first sepconv path (to get 32 channels & same HxW)
    res = SeparableConv2D(32, (1, 5), padding='same', use_bias=False)(inputs)      # Residual features (14x200x32)
    res = BatchNormalization()(res)                                                # BN
    res = Activation('relu')(res)                                                  # ReLU
    print("Residual branch shape:", res.shape)                                     # Shape trace

    # Project x back to 32 channels so we can add with residual
    x = Conv2D(32, (1,1), padding='same', use_bias=False)(x)                       # 1x1 to 32 channels
    x = BatchNormalization()(x)                                                    # BN
    x = Activation('relu')(x)                                                      # ReLU
    print("After 1x1 proj to 32:", x.shape)                                        # Shape trace (1x200x32)

    # Tile the electrode axis from 1 -> 14 to match residual spatial dims for Add()
    x = Lambda(lambda t: tf.repeat(t, repeats=input_shape[0], axis=1))(x)          # Repeat along axis=1
    print("After tiling to electrodes=14:", x.shape)                                # Shape trace (14x200x32)

    # Add residual and continue
    x = Add()([x, res])                                                            # Residual add
    x = BatchNormalization()(x)                                                    # BN
    print("After residual add:", x.shape)                                          # Shape trace

    # First Average Pooling (temporal downsampling: 200 -> 50)
    x = AveragePooling2D((1, 4))(x)                                                # Pool time axis
    x = Dropout(dropout_rate)(x)                                                   # Dropout
    print("After avgpool1:", x.shape)                                              # Shape trace

    # Second separable conv block
    x = SeparableConv2D(64, (1, 3), padding='same', use_bias=False)(x)             # More features
    x = BatchNormalization()(x)                                                    # BN
    x = Activation('relu')(x)                                                      # ReLU
    print("After sepconv2:", x.shape)                                              # Shape trace

    # Second Average Pooling (temporal downsampling: ~50 -> ~12/13)
    x = AveragePooling2D((1, 4))(x)                                                # Pool time again
    x = Dropout(dropout_rate)(x)                                                   # Dropout
    print("After avgpool2:", x.shape)                                              # Shape trace

    # Transformer over electrodes (tokens = 14, features = T' * 64)
    x_shape = K.int_shape(x)                                                       # Static shape
    seq_len = x_shape[1]                                                           # Electrode tokens (14)
    feat_dim = x_shape[2] * x_shape[3]                                             # Feature dim per token
    x = Reshape((seq_len, feat_dim))(x)                                            # (batch, 14, T'*64)
    print("Before Transformer, reshaped to:", x.shape)                              # Trace shape

    x = TransformerBlock(x, num_heads=num_heads, key_dim=64, ff_dim=ff_dim, dropout_rate=0.1)  # Transformer

    # Classifier head
    x = GlobalAveragePooling1D()(x)                                                # Pool across tokens
    x = BatchNormalization()(x)                                                    # BN
    print("After GAP1D:", x.shape)                                                 # Shape trace

    x = Dense(128, activation='relu')(x)                                           # Dense layer
    x = BatchNormalization()(x)                                                    # BN
    x = Dropout(dropout_rate)(x)                                                   # Dropout
    outputs = Dense(1, activation='sigmoid')(x)                                    # Sigmoid for binary classification

    model = Model(inputs, outputs)                                                 # Build model
    return model                                                                   # Return uncompiled model


# ===== Helpers: compile model and callbacks =====

def compile_model():
    """Create and compile a fresh AFNet model instance."""
    model = EEGNet_SpatialTransformer(input_shape=(14, 200, 1))             # Build with correct input shape
    model.compile(optimizer=Adam(learning_rate=1e-3),                       # Adam optimizer (lr=1e-3)
                  loss=BinaryCrossentropy(),                                # Binary cross-entropy loss
                  metrics=[Accuracy()])                                     # Track Accuracy metric
    print(model.summary(line_length=120))                                   # Print model summary
    return model                                                            # Return compiled model

def make_callbacks(fold_idx):
    """Construct callbacks for a given fold: EarlyStopping + ReduceLROnPlateau + ModelCheckpoint(.h5)."""
    print(f"Setting up callbacks for fold {fold_idx}...")                   # Status print

    es = EarlyStopping(                                                     # Early stopping
        monitor='val_loss',                                                 # Monitor validation loss
        patience=15,                                                        # Wait 15 epochs without improvement
        restore_best_weights=True,                                          # Restore best weights
        verbose=1                                                           # Verbose messages
    )

    rlrop = ReduceLROnPlateau(                                              # Reduce LR on plateau
        monitor='val_loss',                                                 # Monitor validation loss
        factor=0.5,                                                         # Halve the LR
        patience=4,                                                         # After 4 stagnant epochs
        min_lr=1e-6,                                                        # Do not go below 1e-6
        verbose=1                                                           # Verbose messages
    )

    ckpt_path = f"best_fold_{fold_idx}.h5"                                  # Save best model as .h5 (HDF5)
    ckpt = ModelCheckpoint(                                                 # Checkpoint callback
        filepath=ckpt_path,                                                 # Output file path
        monitor='val_loss',                                                 # Select best by val_loss
        save_best_only=True,                                                # Only save when improved
        verbose=1                                                           # Verbose messages
        # Note: .h5 requires h5py installed; if missing, run: pip install h5py
    )

    return [es, rlrop, ckpt]                                                # Return callback list


In [10]:
# ===== Step 3 (training): 5-fold CV with per-fold confusion matrix + metrics =====

EPOCHS = 200                                                                   # Upper bound; ES will cut earlier
BATCH_SIZE = 32                                                                # Batch size

per_fold_metrics = []                                                          # List to store per-fold metric dicts
histories = []                                                                 # Store Keras history dicts per fold

fold = 0                                                                       # Initialize fold counter
for train_idx, val_idx in skf.split(X, y):                                     # Iterate stratified folds
    fold += 1                                                                  # Increment fold number

    print("\n" + "="*90)                                                       # Visual separator
    print(f"Fold {fold} — Train/Val split")                                    # Announce fold
    print("Train idx:", train_idx.shape, "Val idx:", val_idx.shape)            # Show split sizes

    X_train, X_val = X[train_idx], X[val_idx]                                  # Slice EEG training/val data
    y_train, y_val = y[train_idx], y[val_idx]                                  # Slice labels training/val

    print("X_train:", X_train.shape, "| y_train:", y_train.shape)              # Shapes
    print("X_val  :", X_val.shape,   "| y_val  :", y_val.shape)                # Shapes

    # ----- Apply per-fold normalization (uses train-only stats) -----
    X_train, X_val = normalize_per_fold(X_train, X_val)                        # Normalize per electrode

    # ✅ Quick sanity check: after normalization, TRAIN should be ~mean=0, std=~1
    train_global_mean = float(np.mean(X_train))
    train_global_std  = float(np.std(X_train))
    val_global_mean   = float(np.mean(X_val))
    val_global_std    = float(np.std(X_val))
    train_per_elec_std = np.std(X_train, axis=(0, 2, 3))  # shape: (14,)

    print(
        "Normalization check — "
        f"Train(mean,std)=({train_global_mean:.4f}, {train_global_std:.4f}) | "
        f"Val(mean,std)=({val_global_mean:.4f}, {val_global_std:.4f}) | "
        f"Train per-electrode std range: [{train_per_elec_std.min():.3f}, {train_per_elec_std.max():.3f}]"
    )

    model = compile_model()                                                    # Build + compile model
    callbacks = make_callbacks(fold)                                           # Create callbacks for this fold

    print("Training with EarlyStopping...")                                    # Status
    history = model.fit(                                                       # Fit model
        X_train, y_train,                                                      # Train data
        validation_data=(X_val, y_val),                                        # Validation data
        epochs=EPOCHS,                                                         # Max epochs
        batch_size=BATCH_SIZE,                                                 # Batch size
        callbacks=callbacks,                                                   # Callbacks list
        verbose=2                                                              # Per-epoch logs
    )

    histories.append(history.history)                                          # Store training history dict

    print("Predicting on validation fold...")                                  # Status
    y_prob = model.predict(X_val, batch_size=BATCH_SIZE).ravel()               # Predict probabilities
    y_pred = (y_prob >= 0.5).astype(int)                                       # Threshold at 0.5 to get classes

    # ----- Confusion Matrix & Per-fold Metrics -----
    cm = confusion_matrix(y_val, y_pred, labels=[0, 1])                        # Confusion matrix
    tn, fp, fn, tp = cm.ravel()                                                # Unpack counts

    print("\nFold", fold, "Confusion Matrix (rows=true, cols=pred) [0=Amateur, 1=Professional]:")
    print(cm)                                                                  # Print matrix
    print(f"TN={tn}, FP={fp}, FN={fn}, TP={tp}")                               # Print counts

    acc  = accuracy_score(y_val, y_pred)                                       # Accuracy
    prec = precision_score(y_val, y_pred, pos_label=1, zero_division=0)        # Precision (class 1)
    sens = recall_score(y_val, y_pred,    pos_label=1, zero_division=0)        # Sensitivity/Recall (class 1)
    spec = tn / (tn + fp) if (tn + fp) > 0 else 0.0                            # Specificity (class 0)
    f1   = f1_score(y_val, y_pred, pos_label=1, zero_division=0)               # F1-score (class 1)

    print("\nFold", fold, "Metrics (threshold=0.5):")                          # Header
    print(f"Accuracy    : {acc:.4f}")                                          # Accuracy
    print(f"Precision   : {prec:.4f}  (class 1: Professional)")                # Precision
    print(f"Sensitivity : {sens:.4f}     (Recall for class 1)")                # Sensitivity/Recall
    print(f"Specificity : {spec:.4f} (for class 0: Amateur)")                  # Specificity
    print(f"F1-score    : {f1:.4f}")                                           # F1

    per_fold_metrics.append({                                                  # Save per-fold metrics
        "fold": fold, "tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp),
        "accuracy": float(acc), "precision": float(prec),
        "sensitivity": float(sens), "specificity": float(spec), "f1": float(f1)
    })

# ----- Per-fold summary printout -----
print("\n=== Per-Fold Metrics Summary ===")                                     # Header
for m in per_fold_metrics:                                                      # Iterate per-fold metrics
    print(f"Fold {m['fold']}: "
          f"Acc={m['accuracy']:.4f}, "
          f"Prec={m['precision']:.4f}, "
          f"Sens={m['sensitivity']:.4f}, "
          f"Spec={m['specificity']:.4f}, "
          f"F1={m['f1']:.4f} | "
          f"TN={m['tn']} FP={m['fp']} FN={m['fn']} TP={m['tp']}")               # Pretty per-fold line

# ----- Optional overall mean ± std across folds -----
accs  = [m["accuracy"]    for m in per_fold_metrics]                            # Collect accuracies
precs = [m["precision"]   for m in per_fold_metrics]                            # Collect precisions
sens  = [m["sensitivity"] for m in per_fold_metrics]                            # Collect sensitivities
specs = [m["specificity"] for m in per_fold_metrics]                            # Collect specificities
f1s   = [m["f1"]          for m in per_fold_metrics]                            # Collect F1 scores

print("\n=== Mean ± Std over 5 folds ===")                                      # Header
print(f"Accuracy    : {np.mean(accs):.4f} ± {np.std(accs):.4f}")                # Mean±Std accuracy
print(f"Precision   : {np.mean(precs):.4f} ± {np.std(precs):.4f}")              # Mean±Std precision
print(f"Sensitivity : {np.mean(sens):.4f} ± {np.std(sens):.4f}")                # Mean±Std sensitivity
print(f"Specificity : {np.mean(specs):.4f} ± {np.std(specs):.4f}")              # Mean±Std specificity
print(f"F1-score    : {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")                  # Mean±Std F1



Fold 1 — Train/Val split
Train idx: (3816,) Val idx: (954,)
X_train: (3816, 14, 200, 1) | y_train: (3816,)
X_val  : (954, 14, 200, 1) | y_val  : (954,)
Per-fold normalization done. train_mean shape: (1, 14, 1, 1) train_std shape: (1, 14, 1, 1)
Normalization check — Train(mean,std)=(-0.0014, 1.0000) | Val(mean,std)=(-0.0077, 1.0277) | Train per-electrode std range: [1.000, 1.000]
Building model with input_shape: (14, 200, 1)
After sepconv1: (None, 14, 200, 32)
After SpatialAttention: (None, 14, 200, 32)
After depthwise spatial: (None, 1, 200, 64)
Residual branch shape: (None, 14, 200, 32)
After 1x1 proj to 32: (None, 1, 200, 32)
After tiling to electrodes=14: (None, 14, 200, 32)
After residual add: (None, 14, 200, 32)
After avgpool1: (None, 14, 50, 32)
After sepconv2: (None, 14, 50, 64)
After avgpool2: (None, 14, 12, 64)
Before Transformer, reshaped to: (None, 14, 768)
After GAP1D: (None, 768)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃ Layer (type)                      ┃ Output Shape                 ┃           Param # ┃ Connected to              
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━
│ input_layer_1 (InputLayer)        │ (None, 14, 200, 1)           │                 0 │ -                         
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ separable_conv2d_3                │ (None, 14, 200, 32)          │                37 │ input_layer_1[0][0]       
│ (SeparableConv2D)                 │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ batch_normalization_8             │ (None, 14, 200, 32)          │               128 │ separable_conv2d_3[0][0]  
│ (BatchNormalization)              │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ activation_5 (Activation)         │ (None, 14, 200, 32)          │                 0 │ batch_normalization_8[0][0
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ global_average_pooling2d_1        │ (None, 32)                   │                 0 │ activation_5[0][0]        
│ (GlobalAveragePooling2D)          │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_6 (Dense)                   │ (None, 64)                   │             2,112 │ global_average_pooling2d_1
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ dense_7 (Dense)                   │ (None, 14)                   │               910 │ dense_6[0][0]             
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ reshape_2 (Reshape)               │ (None, 14, 1, 1)             │                 0 │ dense_7[0][0]             
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ multiply_1 (Multiply)             │ (None, 14, 200, 32)          │                 0 │ activation_5[0][0],       
│                                   │                              │                   │ reshape_2[0][0]           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ depthwise_conv2d_1                │ (None, 1, 200, 64)           │               896 │ multiply_1[0][0]          
│ (DepthwiseConv2D)                 │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ batch_normalization_9             │ (None, 1, 200, 64)           │               256 │ depthwise_conv2d_1[0][0]  
│ (BatchNormalization)              │                              │                   │                           
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ activation_6 (Activation)         │ (None, 1, 200, 64)           │                 0 │ batch_normalization_9[0][0
├───────────────────────────────────┼──────────────────────────────┼───────────────────┼───────────────────────────
│ conv2d_1 (Conv2D)                 │ (None, 1, 200, 32)           │             2,048 │ activation_6[0][0]        
├───────────────────────────────────┼───────────────────

 Total params: 1,099,897 (4.20 MB)

 Trainable params: 1,097,593 (4.19 MB)

...

120/120 - 21s - 174ms/step - accuracy: 2.6205e-04 - loss: 0.1666 - val_accuracy: 0.0000e+00 - val_loss: 1.5765 - learning_rate: 1.0000e-03
Epoch 3/200

Epoch 3: val_loss improved from 1.57647 to 0.43208, saving model to best_fold_5.h5


120/120 - 21s - 173ms/step - accuracy: 5.2411e-04 - loss: 0.1157 - val_accuracy: 0.0000e+00 - val_loss: 0.4321 - learning_rate: 1.0000e-03
Epoch 4/200

Epoch 4: val_loss improved from 0.43208 to 0.07358, saving model to best_fold_5.h5


120/120 - 21s - 174ms/step - accuracy: 7.8616e-04 - loss: 0.0946 - val_accuracy: 0.0000e+00 - val_loss: 0.0736 - learning_rate: 1.0000e-03
Epoch 5/200

Epoch 5: val_loss did not improve from 0.07358
120/120 - 21s - 172ms/step - accuracy: 0.0029 - loss: 0.0851 - val_accuracy: 0.0000e+00 - val_loss: 0.1165 - learning_rate: 1.0000e-03
Epoch 6/200

Epoch 6: val_loss improved from 0.07358 to 0.06515, saving model to best_fold_5.h5


120/120 - 21s - 175ms/step - accuracy: 0.0026 - loss: 0.0742 - val_accuracy: 0.0010 - val_loss: 0.0651 - learning_rate: 1.0000e-03
Epoch 7/200

Epoch 7: val_loss did not improve from 0.06515
120/120 - 21s - 174ms/step - accuracy: 0.0018 - loss: 0.0650 - val_accuracy: 0.0199 - val_loss: 0.1760 - learning_rate: 1.0000e-03
Epoch 8/200

Epoch 8: val_loss did not improve from 0.06515
120/120 - 21s - 174ms/step - accuracy: 0.0031 - loss: 0.0676 - val_accuracy: 0.0294 - val_loss: 0.1342 - learning_rate: 1.0000e-03
Epoch 9/200

Epoch 9: val_loss did not improve from 0.06515
120/120 - 21s - 175ms/step - accuracy: 0.0037 - loss: 0.0558 - val_accuracy: 0.0524 - val_loss: 0.1070 - learning_rate: 1.0000e-03
Epoch 10/200

Epoch 10: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 10: val_loss did not improve from 0.06515
120/120 - 21s - 174ms/step - accuracy: 0.0016 - loss: 0.0526 - val_accuracy: 0.0241 - val_loss: 0.0732 - learning_rate: 1.0000e-03
Epoch 11/200

Epoch 11: v

120/120 - 21s - 174ms/step - accuracy: 0.0071 - loss: 0.0347 - val_accuracy: 0.0356 - val_loss: 0.0552 - learning_rate: 5.0000e-04
Epoch 12/200

Epoch 12: val_loss did not improve from 0.05524
120/120 - 21s - 172ms/step - accuracy: 0.0084 - loss: 0.0271 - val_accuracy: 0.0860 - val_loss: 0.0569 - learning_rate: 5.0000e-04
Epoch 13/200

Epoch 13: val_loss did not improve from 0.05524
120/120 - 21s - 173ms/step - accuracy: 0.0110 - loss: 0.0223 - val_accuracy: 0.0482 - val_loss: 0.0658 - learning_rate: 5.0000e-04
Epoch 14/200

Epoch 14: val_loss did not improve from 0.05524
120/120 - 21s - 173ms/step - accuracy: 0.0118 - loss: 0.0203 - val_accuracy: 0.0765 - val_loss: 0.0827 - learning_rate: 5.0000e-04
Epoch 15/200

Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 15: val_loss did not improve from 0.05524
120/120 - 21s - 172ms/step - accuracy: 0.0168 - loss: 0.0167 - val_accuracy: 0.1593 - val_loss: 0.0795 - learning_rate: 5.0000e-04
Epoch 16/200

Epoch